In [5]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go

In [2]:
project_name = 'Split_Sys_VQA'
# project_name = 'Split_Sub_VQA'

# Show Continual Training Results

In [7]:
def get_accs(exp_name):
    results = np.load(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{exp_name}.npy'), allow_pickle=True)
    accs = [results[i]['Top1_Acc_Stream/eval_phase/test_stream/Task000'] for i in range(len(results))]
    losses = [results[i]['Loss_Stream/eval_phase/test_stream/Task000'] for i in range(len(results))]
    return accs, losses

In [8]:
exp_names = ['Naive-lr0_005', 'ER-lr0_01-m1000', 'ER-lr0_01-m5000', 'LwF-lr0_005-a11-t2', 'GEM-lr0_01-p32-m0_5']
dsp_names = ['Naive', 'ER-1000', 'ER-5000', 'LwF', 'GEM']
all_accs = [get_accs(exp_names[i])[0] for i in range(len(exp_names))]
all_losses = [get_accs(exp_names[i])[1] for i in range(len(exp_names))]

In [14]:
collected_data = []
for idx in range(len(exp_names)):
    collected_data.append(pd.DataFrame({'Num classes': np.arange(2, 21, 2), 'Algo.': dsp_names[idx], 'Type': 'Acc', 'Value': all_accs[idx]}))

'''add RPSnet'''
collected_data.append(pd.DataFrame({
    'Num classes': np.arange(2, 21, 2), 'Algo.': 'RPSnet', 'Type': 'Acc',
    'Value': np.array([74.5, 68.75, 58.166667, 50.875, 47.4, 43.5, 35.785714, 34.4375, 22.277778, 20.15])/100
}))

collected_data = pd.concat(collected_data, ignore_index=True)

fig = px.line(collected_data, x="Num classes", y="Value", color="Algo.", title='Acc')
fig.show()
# write_image(fig, experiment_name, "train.png")

In [15]:
collected_data = []
for idx in range(len(exp_names)):
    collected_data.append(pd.DataFrame({'Num classes': np.arange(2, 21, 2), 'Algo.': dsp_names[idx], 'Type': 'Loss', 'Value': all_losses[idx]}))


collected_data = pd.concat(collected_data, ignore_index=True)

fig = px.line(collected_data, x="Num classes", y="Value", color="Algo.", title='Loss')
fig.show()
# write_image(fig, experiment_name, "train.png")

# Show Novel and Non-novel Testing Results

In [40]:

# exp_name = 'Naive-lr0_005'       # nc-
# exp_name = 'ER-lr0_01-m1000'
# exp_name = 'LwF-lr0_005-a11-t2'
exp_name = 'GEM-lr0_01-p32-m0_5'        # GEM-lr0_01-p32-m0_5 color-GEM-lr0_01-p32-m0_3

# test_case_name = 'Novel-{}'.format(exp_name)
# test_case_name = 'Non-novel-{}'.format(exp_name)
# test_case_name = 'Novel-{}-frz'.format(exp_name)
test_case_name = 'Non-novel-{}-frz'.format(exp_name)
# test_case_name = 'Novel-{}-lm'.format(exp_name)
# test_case_name = 'Novel-{}-frz-lm'.format(exp_name)

In [41]:
results = np.load(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{test_case_name}.npy'), allow_pickle=True)

In [42]:
'''Test acc'''
accs = [result['Top1_Acc_Stream/eval_phase/test_stream/Task000'] for result in results]
print(accs)

[0.45, 0.53, 0.4, 0.41, 0.49, 0.61, 0.39, 0.51, 0.57, 0.44, 0.48, 0.57, 0.66, 0.53, 0.55, 0.53, 0.6, 0.5, 0.6, 0.37, 0.54, 0.55, 0.4, 0.48, 0.51, 0.48, 0.56, 0.42, 0.47, 0.44, 0.42, 0.6, 0.52, 0.34, 0.63, 0.51, 0.35, 0.62, 0.5, 0.58, 0.53, 0.61, 0.52, 0.46, 0.47, 0.48, 0.56, 0.52, 0.36, 0.58, 0.4, 0.46, 0.41, 0.55, 0.42, 0.46, 0.56, 0.44, 0.53, 0.5, 0.59, 0.41, 0.49, 0.52, 0.39, 0.48, 0.46, 0.43, 0.37, 0.5, 0.4, 0.46, 0.53, 0.46, 0.41, 0.29, 0.49, 0.49, 0.45, 0.32, 0.48, 0.38, 0.48, 0.47, 0.45, 0.69, 0.44, 0.5, 0.59, 0.48, 0.48, 0.35, 0.53, 0.44, 0.53, 0.53, 0.42, 0.64, 0.52, 0.44, 0.55, 0.36, 0.64, 0.61, 0.65, 0.44, 0.47, 0.47, 0.49, 0.5, 0.43, 0.58, 0.5, 0.46, 0.54, 0.55, 0.49, 0.47, 0.64, 0.5, 0.46, 0.5, 0.49, 0.46, 0.3, 0.58, 0.55, 0.49, 0.63, 0.58, 0.43, 0.35, 0.39, 0.59, 0.45, 0.49, 0.57, 0.38, 0.55, 0.47, 0.58, 0.38, 0.52, 0.45, 0.51, 0.48, 0.37, 0.52, 0.63, 0.58, 0.51, 0.54, 0.59, 0.4, 0.45, 0.44, 0.36, 0.44, 0.41, 0.58, 0.4, 0.5, 0.53, 0.53, 0.59, 0.49, 0.39, 0.52, 0.49, 0.5, 

In [43]:
print(f'{np.mean(accs)*100:.2f}% ({np.std(accs)*100:.2f}%)')

49.83% (8.51%)


# Show Hyper-Parameter Tuning Results

In [33]:
def display_results(exp_name, test_case_name):
    print(f'{exp_name}, {test_case_name}:')
    if os.path.exists(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{test_case_name}.npy')):
        results = np.load(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{test_case_name}.npy'), allow_pickle=True)
    else:
        print(f'results-{test_case_name}.npy does not exist.')
        return
    accs = [result['Top1_Acc_Stream/eval_phase/test_stream/Task000'] for result in results]
    print(f'==> {accs}')
    print(f'==> {np.mean(accs)*100:.2f}% ({np.std(accs)*100:.2f}%)')

In [5]:
exp_names = [
    'Naive-lr0_001', 'Naive-lr0_005', 'Naive-lr0_01', 'Naive-lr0_05', 'Naive-lr0_1',
]
exp_names = [f'color-{exp_names[i]}' for i in range(len(exp_names))]

for exp_name in exp_names:
    display_results(exp_name, exp_name)

color-Naive-lr0_001, color-Naive-lr0_001:
==> [0.97, 0.48, 0.30333333333333334, 0.2375, 0.178, 0.16, 0.13428571428571429, 0.10625, 0.10333333333333333, 0.092]
==> 27.65% (25.71%)
color-Naive-lr0_005, color-Naive-lr0_005:
==> [0.98, 0.475, 0.3, 0.24, 0.174, 0.15, 0.14, 0.11125, 0.10666666666666667, 0.093]
==> 27.70% (25.91%)
color-Naive-lr0_01, color-Naive-lr0_01:
==> [0.96, 0.465, 0.30666666666666664, 0.2425, 0.172, 0.145, 0.13714285714285715, 0.115, 0.10333333333333333, 0.091]
==> 27.38% (25.35%)
color-Naive-lr0_05, color-Naive-lr0_05:
==> [0.95, 0.47, 0.30333333333333334, 0.245, 0.168, 0.16, 0.13857142857142857, 0.10875, 0.09888888888888889, 0.092]
==> 27.35% (25.11%)
color-Naive-lr0_1, color-Naive-lr0_1:
==> [0.98, 0.47, 0.31, 0.2375, 0.162, 0.15333333333333332, 0.14, 0.1075, 0.09888888888888889, 0.091]
==> 27.50% (26.01%)


In [37]:
exp_names = [
    'ER-lr0_001-m100', 'ER-lr0_01-m100', 'ER-lr0_1-m100',
    'ER-lr0_001-m500', 'ER-lr0_01-m500', 'ER-lr0_1-m500',
    'ER-lr0_001-m1000', 'ER-lr0_01-m1000', 'ER-lr0_1-m1000',
    'ER-lr0_001-m5000', 'ER-lr0_01-m5000', 'ER-lr0_1-m5000',
]
exp_names = [f'color-{exp_names[i]}' for i in range(len(exp_names))]

for exp_name in exp_names:
    display_results(exp_name, exp_name)

color-ER-lr0_001-m100, color-ER-lr0_001-m100:
==> [0.97, 0.695, 0.45666666666666667, 0.2925, 0.222, 0.18, 0.15428571428571428, 0.125, 0.11333333333333333, 0.094]
==> 33.03% (27.75%)
color-ER-lr0_01-m100, color-ER-lr0_01-m100:
==> [0.96, 0.71, 0.4633333333333333, 0.3025, 0.24, 0.175, 0.16857142857142857, 0.13625, 0.12333333333333334, 0.092]
==> 33.71% (27.47%)
color-ER-lr0_1-m100, color-ER-lr0_1-m100:
==> [0.98, 0.635, 0.37333333333333335, 0.285, 0.23, 0.20333333333333334, 0.16142857142857142, 0.13, 0.12222222222222222, 0.095]
==> 32.15% (26.68%)
color-ER-lr0_001-m500, color-ER-lr0_001-m500:
==> [0.97, 0.805, 0.6, 0.33, 0.362, 0.3016666666666667, 0.21285714285714286, 0.1775, 0.18777777777777777, 0.109]
==> 40.56% (27.55%)
color-ER-lr0_01-m500, color-ER-lr0_01-m500:
==> [0.96, 0.78, 0.61, 0.3775, 0.36, 0.36666666666666664, 0.23714285714285716, 0.21, 0.1988888888888889, 0.127]
==> 42.27% (26.03%)
color-ER-lr0_1-m500, color-ER-lr0_1-m500:
results-color-ER-lr0_1-m500.npy does not exist.
col

In [38]:
exp_names = [
    'LwF-lr0_001-a9-t2', 'LwF-lr0_005-a9-t2', 'LwF-lr0_01-a9-t2', 'LwF-lr0_05-a9-t2',  'LwF-lr0_1-a9-t2',
    # 'LwF-lr0_001-a10-t2', 'LwF-lr0_005-a10-t2', 'LwF-lr0_01-a10-t2', 'LwF-lr0_05-a10-t2',  'LwF-lr0_1-a10-t2',
    'LwF-lr0_001-a11-t2', 'LwF-lr0_005-a11-t2', 'LwF-lr0_01-a11-t2', 'LwF-lr0_05-a11-t2',  'LwF-lr0_1-a11-t2',
]
# exp_names = [
#     'LwF-lr0_005-a5-t1','LwF-lr0_005-a10-t1', 'LwF-lr0_005-a15-t1', 'LwF-lr0_005-a20-t1', 'LwF-lr0_005-a25-t1',
#     'LwF-lr0_005-a5-t2','LwF-lr0_005-a10-t2', 'LwF-lr0_005-a15-t2', 'LwF-lr0_005-a20-t2', 'LwF-lr0_005-a25-t2',
#     'LwF-lr0_005-a5-t3','LwF-lr0_005-a10-t3', 'LwF-lr0_005-a15-t3', 'LwF-lr0_005-a20-t3', 'LwF-lr0_005-a25-t3',
#     'LwF-lr0_005-a5-t4','LwF-lr0_005-a10-t4', 'LwF-lr0_005-a15-t4', 'LwF-lr0_005-a20-t4', 'LwF-lr0_005-a25-t4',
#     'LwF-lr0_005-a5-t5','LwF-lr0_005-a10-t5', 'LwF-lr0_005-a15-t5', 'LwF-lr0_005-a20-t5', 'LwF-lr0_005-a25-t5',
# ]
exp_names = [f'color-{exp_names[i]}' for i in range(len(exp_names))]

for exp_name in exp_names:
    display_results(exp_name, exp_name)

color-LwF-lr0_001-a9-t2, color-LwF-lr0_001-a9-t2:
==> [0.97, 0.445, 0.3, 0.24, 0.162, 0.15, 0.13857142857142857, 0.10875, 0.10555555555555556, 0.093]
==> 27.13% (25.50%)
color-LwF-lr0_005-a9-t2, color-LwF-lr0_005-a9-t2:
==> [0.98, 0.445, 0.3, 0.2375, 0.178, 0.15166666666666667, 0.14142857142857143, 0.11125, 0.10444444444444445, 0.094]
==> 27.43% (25.67%)
color-LwF-lr0_01-a9-t2, color-LwF-lr0_01-a9-t2:
results-color-LwF-lr0_01-a9-t2.npy does not exist.
color-LwF-lr0_05-a9-t2, color-LwF-lr0_05-a9-t2:
results-color-LwF-lr0_05-a9-t2.npy does not exist.
color-LwF-lr0_1-a9-t2, color-LwF-lr0_1-a9-t2:
results-color-LwF-lr0_1-a9-t2.npy does not exist.
color-LwF-lr0_001-a11-t2, color-LwF-lr0_001-a11-t2:
==> [0.97, 0.435, 0.30333333333333334, 0.2425, 0.164, 0.15833333333333333, 0.13857142857142857, 0.1075, 0.10555555555555556, 0.093]
==> 27.18% (25.39%)
color-LwF-lr0_005-a11-t2, color-LwF-lr0_005-a11-t2:
results-color-LwF-lr0_005-a11-t2.npy does not exist.
color-LwF-lr0_01-a11-t2, color-LwF-lr0_0

In [39]:
exp_names = [
    'GEM-lr0_01-p16-m0_3', 'GEM-lr0_01-p32-m0_3',
    'GEM-lr0_01-p16-m0_5', 'GEM-lr0_01-p32-m0_5',

    'GEM-lr0_1-p16-m0_3', 'GEM-lr0_1-p32-m0_3',
    'GEM-lr0_1-p16-m0_5', 'GEM-lr0_1-p32-m0_5',
]
exp_names = [f'color-{exp_names[i]}' for i in range(len(exp_names))]

for exp_name in exp_names:
    display_results(exp_name, exp_name)

color-GEM-lr0_01-p16-m0_3, color-GEM-lr0_01-p16-m0_3:
==> [0.96, 0.515, 0.35, 0.2675, 0.152, 0.19666666666666666, 0.15285714285714286, 0.12, 0.11, 0.104]
==> 29.28% (25.42%)
color-GEM-lr0_01-p32-m0_3, color-GEM-lr0_01-p32-m0_3:
==> [0.96, 0.585, 0.41, 0.2725, 0.204, 0.225, 0.17714285714285713, 0.12625, 0.12, 0.102]
==> 31.82% (25.65%)
color-GEM-lr0_01-p16-m0_5, color-GEM-lr0_01-p16-m0_5:
==> [0.96, 0.49, 0.31, 0.2775, 0.154, 0.14666666666666667, 0.14142857142857143, 0.115, 0.12111111111111111, 0.089]
==> 28.05% (25.48%)
color-GEM-lr0_01-p32-m0_5, color-GEM-lr0_01-p32-m0_5:
results-color-GEM-lr0_01-p32-m0_5.npy does not exist.
color-GEM-lr0_1-p16-m0_3, color-GEM-lr0_1-p16-m0_3:
results-color-GEM-lr0_1-p16-m0_3.npy does not exist.
color-GEM-lr0_1-p32-m0_3, color-GEM-lr0_1-p32-m0_3:
results-color-GEM-lr0_1-p32-m0_3.npy does not exist.
color-GEM-lr0_1-p16-m0_5, color-GEM-lr0_1-p16-m0_5:
results-color-GEM-lr0_1-p16-m0_5.npy does not exist.
color-GEM-lr0_1-p32-m0_5, color-GEM-lr0_1-p32-m0_5: